In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

In [ ]:
def show_scrollable(df, max_rows=20, height=400):
    html = df.head(max_rows).to_html(notebook=True)
    display(HTML(f'<div style="overflow:auto; height:{height}px">{html}</div>'))

In [ ]:
df_session = pd.read_csv('Data\TuSession_Nicola.csv', encoding='latin1')
after_session = pd.read_csv('Data\Efterkodsession_Nicola.csv', encoding='latin1')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\E'
C:\Users\nicoc\AppData\Local\Temp\ipykernel_2288\1617187588.py:1: SyntaxWarning: invalid escape sequence '\T'
  df_session = pd.read_csv('Data\TuSession_Nicola.csv', encoding='latin1')
C:\Users\nicoc\AppData\Local\Temp\ipykernel_2288\1617187588.py:2: SyntaxWarning: invalid escape sequence '\E'
  after_session = pd.read_csv('Data\Efterkodsession_Nicola.csv', encoding='latin1')


In [ ]:
df_session.columns

Index(['SessionId', 'interviewtrafikdato', 'interviewtype', 'teleintno',
       'interviewsessionslutflag', 'skemaversion', 'boligandenadrflag',
       'boligandenadrsogeord', 'boligandenadrid', 'boligandenadrstedid',
       'boligandenadrtext', 'arbpladrsogeord', 'arbpladrid', 'arbpladrstedid',
       'arbpladrtext', 'PrimOccMuncode', 'udgptype', 'udgpadrsogeord',
       'udgpadrid', 'udgpadrstedid', 'udgpadrtext', 'startstedflag',
       'startstedadrsogeord', 'startstedadrid', 'startstedadrstedid',
       'startstedadrtext', 'startstedadre', 'startstedadrn', 'startstedkomnr',
       'startformaal'],
      dtype='object')

In [ ]:
df_session.dtypes

SessionId                  int64
homeID                     Int64
alternativehomeID          int64
homeText                  object
arbpladrsogeord           object
workID                     Int64
alternativeworkID          Int64
workText                  object
workMunCode                Int64
udgptype                   Int64
udgpadrsogeord            object
startDayID                 Int64
alternativeStartDayID      Int64
startDayText              object
startstedflag              int64
startstedadrsogeord       object
startTripID                Int64
alternativeStartTripID     int64
startTripText             object
startstedadre              int64
startstedadrn              int64
startstedkomnr             int64
homeadrmuncode             int64
dtype: object

In [ ]:
after_session.dtypes

In [ ]:
after_session.columns

Index(['SessionId', 'medtagsession', 'boligandenadrid', 'boligandenadrstedid',
       'boligandenadrtext', 'homeadre', 'homeadrn', 'homeadrmuncode',
       'boligandenadrkonklu', 'arbpladrid', 'arbpladrstedid', 'arbpladrtext',
       'workple', 'workpln', 'workplmuncode', 'arbplkonklu', 'udgpadrid',
       'udgpadrstedid', 'udgpadrtext', 'sduadre', 'sduadrn', 'sduadrmuncode',
       'udgpkonklu', 'startstedadrid', 'startstedadrstedid',
       'startstedadrtext', 'startstedadre', 'startstedadrn', 'daystartmuncode',
       'startstedkonklu'],
      dtype='object')

In [ ]:
afterdf = after_session 

In [ ]:
df_session.drop(columns=['interviewtrafikdato', 'interviewtype', 'teleintno', 'interviewsessionslutflag','skemaversion','boligandenadrflag','startformaal','boligandenadrsogeord'], inplace=True)

In [ ]:
# Get the order of columns in df_session that are also in after_session
common_ordered = [col for col in df_session.columns if col in after_session.columns]

# Get columns that are only in after_session
after_only = [col for col in after_session.columns if col not in df_session.columns]

# Reorder after_session columns to match df_session order, then add unique columns
after_session = after_session[common_ordered + after_only]

In [ ]:
df_session['udgptype'] = pd.to_numeric(df_session['udgptype'], errors='coerce').astype('Int64')

# after_session.rename(columns={'daystartmuncode':'startstedkomnr', 'workplmuncode':'workMunCode'}, inplace=True)

#"sduadre","sduadrn" are the UTM coords of start of the day

df_session.rename(columns=
                {'boligandenadrid':'homeID',
                 'boligandenadrstedid':'alternativehomeID',
                 'boligandenadrtext':'homeText',
                 'arbpladrid':'workID',
                 'arbpladrstedid':'alternativeworkID',
                 'arbpladrtext':'workText',
                 'PrimOccMuncode':'workMunCode',
                 'udgpadrid':'startDayID', # where do you sleep usually in weekdays
                 'udgpadrstedid':'alternativeStartDayID',
                 'udgpadrtext':'startDayText',
                 'startstedadrid':'startTripID', # where you started yesterday trip
                 'startstedadrstedid':'alternativeStartTripID',
                 'startstedadrtext':'startTripText'
                 }, inplace=True)

after_session.rename(columns=
                {'boligandenadrid':'homeID',
                 'boligandenadrstedid':'alternativehomeID',
                 'boligandenadrtext':'homeText',
                 'arbpladrid':'workID',
                 'arbpladrstedid':'alternativeworkID',
                 'arbpladrtext':'workText',
                 'PrimOccMuncode':'workMunCode',
                 'udgpadrid':'startDayID', # where do you sleep usually in weekdays
                 'udgpadrstedid':'alternativeStartDayID',
                 'udgpadrtext':'startDayText',
                 'startstedadrid':'startTripID', # where you started yesterday trip
                 'startstedadrstedid':'alternativeStartTripID',
                 'startstedadrtext':'startTripText'
                 }, inplace=True)

In [ ]:
# Drop columns ending with 'konklu' from after_session
cols_to_drop = [col for col in after_session.columns if col.endswith('konklu')]
after_session.drop(columns=cols_to_drop, inplace=True)

df_session['homeadrmuncode'] = df_session['homeID'].str[:3].astype(int)

In [ ]:
dupes = after_session.columns[after_session.columns.duplicated()].unique()
print("Duplicate columns found:", list(dupes))

# Merge duplicate pairs (keep first occurrence, fill NA from later)
for col in dupes:
    # Get positions of all columns with this name
    col_positions = [i for i, c in enumerate(after_session.columns) if c == col]
    if len(col_positions) > 1:
        print(f"→ Unifying duplicates for '{col}' ({len(col_positions)} copies)")
        
        # Get the first column by position (this ensures we get a Series)
        merged = after_session.iloc[:, col_positions[0]].copy()
        
        # Combine with other columns with the same name
        for pos in col_positions[1:]:
            other_col = after_session.iloc[:, pos]
            merged = merged.combine_first(other_col)
        
        # Replace the first occurrence with merged data
        after_session.iloc[:, col_positions[0]] = merged

# Remove all duplicate columns, keeping only the first (now merged) occurrence
after_session = after_session.loc[:, ~after_session.columns.duplicated()]

# Confirm cleanup
print("After cleanup, any duplicates?", after_session.columns.duplicated().any())

Duplicate columns found: []
After cleanup, any duplicates? False


In [ ]:
after_session.columns

Index(['SessionId', 'homeID', 'alternativehomeID', 'homeText', 'workID',
       'alternativeworkID', 'workText', 'startDayID', 'alternativeStartDayID',
       'startDayText', 'startTripID', 'alternativeStartTripID',
       'startTripText', 'startstedadre', 'startstedadrn', 'medtagsession',
       'homeadre', 'homeadrn', 'homeadrmuncode', 'workple', 'workpln',
       'workplmuncode', 'sduadre', 'sduadrn', 'sduadrmuncode',
       'daystartmuncode'],
      dtype='object')

In [ ]:
cols_to_int = ['homeID', 'workID', 'alternativeworkID', 'workMunCode', 'startDayID','alternativeStartDayID','startTripID']
for col in cols_to_int:
    if col in df_session.columns:
        df_session[col] = pd.to_numeric(df_session[col], errors='coerce').astype('Int64')

# Merge raw and modified session tables
s = df_session.merge(after_session, on="SessionId", suffixes=("_raw","_mod"), how="inner")

# Identify comparable columns (those present in both, excluding key)
sess_common_cols = sorted(set(df_session.columns).intersection(after_session.columns))
sess_compare_cols = [c for c in sess_common_cols if c != "SessionId"]

print("Merged shape:", s.shape)
print("Comparable columns:", sess_compare_cols[:10], " ... total:", len(sess_compare_cols))

Merged shape: (74596, 48)
Comparable columns: ['alternativeStartDayID', 'alternativeStartTripID', 'alternativehomeID', 'alternativeworkID', 'homeID', 'homeText', 'homeadrmuncode', 'startDayID', 'startDayText', 'startTripID']  ... total: 15


In [ ]:
session_only_cols = sorted(set(df_session.columns) - set(after_session.columns))
after_only_cols = sorted(set(after_session.columns) - set(df_session.columns))

print("Columns only in df_session (not in after_session):")
print(session_only_cols)

print("\nColumns only in after_session (not in df_session):")
print(after_only_cols)

Columns only in df_session (not in after_session):
['arbpladrsogeord', 'startstedadrsogeord', 'startstedflag', 'startstedkomnr', 'udgpadrsogeord', 'udgptype', 'workMunCode']

Columns only in after_session (not in df_session):
['daystartmuncode', 'homeadre', 'homeadrn', 'medtagsession', 'sduadre', 'sduadrmuncode', 'sduadrn', 'workple', 'workplmuncode', 'workpln']


In [ ]:
if not s.index.is_unique:
    print("Index not unique — resetting index.")
s = s.reset_index(drop=True)

# 1) Per-column change masks: (mod is not NA) AND (mod != raw)
sess_diff_masks = {}
for c in sess_compare_cols:
    eq = s[f"{c}_mod"].eq(s[f"{c}_raw"])                  # may contain <NA>
    changed = s[f"{c}_mod"].notna() & (~eq.fillna(False))  # strictly boolean
    # store as a plain numpy array to avoid any alignment surprises
    sess_diff_masks[c] = changed.to_numpy()

# 2) Assemble boolean matrix with the same row count as `s`
sess_diff_df = pd.DataFrame(sess_diff_masks, index=s.index).astype(bool)

# 3) Row-wise list/count of changed columns + flag
s["changed_cols_session"] = sess_diff_df.apply(lambda r: [k for k, v in r.items() if v], axis=1)
s["n_changed_session"]    = sess_diff_df.sum(axis=1).astype("int64")
s["flag_session"]         = np.where(s["n_changed_session"] > 0, "WRONG", "OK")

In [ ]:
s.columns 

Index(['SessionId', 'homeID_raw', 'alternativehomeID_raw', 'homeText_raw',
       'arbpladrsogeord', 'workID_raw', 'alternativeworkID_raw',
       'workText_raw', 'workMunCode', 'udgptype', 'udgpadrsogeord',
       'startDayID_raw', 'alternativeStartDayID_raw', 'startDayText_raw',
       'startstedflag', 'startstedadrsogeord', 'startTripID_raw',
       'alternativeStartTripID_raw', 'startTripText_raw', 'startstedadre_raw',
       'startstedadrn_raw', 'startstedkomnr', 'homeadrmuncode_raw',
       'homeID_mod', 'alternativehomeID_mod', 'homeText_mod', 'workID_mod',
       'alternativeworkID_mod', 'workText_mod', 'startDayID_mod',
       'alternativeStartDayID_mod', 'startDayText_mod', 'startTripID_mod',
       'alternativeStartTripID_mod', 'startTripText_mod', 'startstedadre_mod',
       'startstedadrn_mod', 'medtagsession', 'homeadre', 'homeadrn',
       'homeadrmuncode_mod', 'workple', 'workpln', 'workplmuncode', 'sduadre',
       'sduadrn', 'sduadrmuncode', 'daystartmuncode', 'change

In [ ]:
sess_edit = s[['SessionId', 'homeText_raw','homeadre','homeadrn','workText_raw','workText_mod','workple','workpln','startDayText_raw','startDayText_mod','startstedadrsogeord','startTripText_raw','startTripText_mod', 'startstedadre_raw',
       'startstedadrn_raw','startstedadre_mod',
       'startstedadrn_mod','daystartmuncode']]

In [ ]:
sess_edit.dtypes

Index(['SessionId', 'homeText_raw', 'homeadre', 'homeadrn', 'workText_raw',
       'workText_mod', 'workple', 'workpln', 'startDayText_raw',
       'startDayText_mod', 'startstedadrsogeord', 'startTripText_raw',
       'startTripText_mod', 'startstedadre_raw', 'startstedadrn_raw',
       'startstedadre_mod', 'startstedadrn_mod', 'daystartmuncode'],
      dtype='object')

: 

## Here we explicitly change the value for the start of the trip, to start of the day, to the rows that have a 999 value in daystartmuncode, which means they inputted a location outside of denmark

In [ ]:
sess_edit['startTripText_raw'] = np.where(sess_edit['daystartmuncode'] == 999, sess_edit['startDayText_raw'], sess_edit['startTripText_raw'])

C:\Users\nicoc\AppData\Local\Temp\ipykernel_2288\2645171465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sess_edit['startTripText_raw'] = np.where(sess_edit['daystartmuncode'] == 999, sess_edit['startDayText_raw'], sess_edit['startTripText_raw'])


In [ ]:
sess_edit.to_csv('Data/TuSession_Edited_Nicola.csv', index=False, encoding='latin1')